In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sample-submission/sample_submission-v0.2.csv
/kaggle/input/test-ranking-public/data/processed/public/task_1_query-product_ranking/test_public-v0.2.csv
/kaggle/input/product-catalogue/data/processed/public/task_1_query-product_ranking/product_catalogue-v0.2.csv
/kaggle/input/sentence-transformer-classification-model/config.json
/kaggle/input/sentence-transformer-classification-model/__results__.html
/kaggle/input/sentence-transformer-classification-model/README.md
/kaggle/input/sentence-transformer-classification-model/tokenizer.json
/kaggle/input/sentence-transformer-classification-model/tokenizer_config.json
/kaggle/input/sentence-transformer-classification-model/sentence_bert_config.json
/kaggle/input/sentence-transformer-classification-model/pytorch_model.bin
/kaggle/input/sentence-transformer-classification-model/__notebook__.ipynb
/kaggle/input/sentence-transformer-classification-model/config_sentence_transformers.json
/kaggle/input/sentence-transformer-classificatio

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

In [3]:
test_df = pd.read_csv('../input/test-ranking-public/data/processed/public/task_1_query-product_ranking/test_public-v0.2.csv')

In [4]:
prod_cat = pd.read_csv('../input/product-catalogue/data/processed/public/task_1_query-product_ranking/product_catalogue-v0.2.csv')

In [5]:
prod_cat.head()
prod_cat.product_id.value_counts()
# repeating prod ids across 3 locales

B07Q28DSYL    3
B072J8XD3D    3
B00P8XQPY4    3
B07GK35P2C    3
B00MH4QKP6    3
             ..
B07WKKDCPT    1
B08KT518JX    1
B001E1H7QY    1
B005ACH8SU    1
B09CNNHRQ1    1
Name: product_id, Length: 879290, dtype: int64

In [6]:
query_prod_test = pd.merge(test_df, prod_cat, right_on=['product_id','product_locale'], left_on=['product_id', 'query_locale'], suffixes=('_test','_catlog'), how='left')

In [7]:
def get_query_input(row):
    query_info = str(row.get('query', '')) + ' ' + str(row.get('query_locale', ''))
    return query_info.strip()

d = {'query': 'hi hello',}
assert get_query_input(d) == 'hi hello'
d = {'query': 'hi hello', 'query_locale': 'en'}
assert get_query_input(d) == 'hi hello en'

In [8]:
query_prod_test.fillna('', inplace=True)
query_prod_test['query_input'] = query_prod_test.apply(lambda row: get_query_input(row), axis=1)

In [9]:
def get_product_input(row):
    product_columns = ['product_title', 'product_locale', 'product_description']
    product_info = ''
    for col in product_columns:
        product_info += str(row.get(col, '')) + ' '
    return product_info.strip()

d = {'product_title': 'hi hello', 'product_locale': 'en', 'product_description': 'nike'}
assert get_product_input(d) == 'hi hello en nike'

In [10]:
query_prod_test['product_input'] = query_prod_test.apply(lambda row: get_product_input(row), axis=1)

In [11]:
scores = {'exact': 1.0, 'substitute': 0.1, 'complement': 0.01, 'irrelevant': 0.0}

In [12]:
query_prod_test.head()

,query_id,query,query_locale,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale,query_input,product_input
0,33777,!qscreen fence without holes,us,B000HMCA7W,"Tenax 60041989 Multi-Purpose Net, 3' x 50', Black",,"Mesh .70""X.98""\nVirtually invisible\nWill not rust or corrode\nAvailable in black",Tenax,Black,us,!qscreen fence without holes us,"Tenax 60041989 Multi-Purpose Net, 3' x 50', Black us"
1,33777,!qscreen fence without holes,us,B0019Y6TTC,Garden Creations JB4710 Extendable Instant Fence,,"EXPANDABLE FLEXI-SCREEN stretch your fence to the width compatable with your home\nESTHETIC DESIGN easily place your fence inside or outdoors, wood finish is a perfect match for interior home designs as well as porch finishes\nPLANT DIVIDER perfect for flowerbeds, easily separate your plants while maintaining visual garden appeal\nPORTABLE light weight design, easy to move from room to room\nDURABLE fence stands on two wooden platforms, no digging or support needed","Creation's Garden Natural Products, Inc",Brown,us,!qscreen fence without holes us,Garden Creations JB4710 Extendable Instant Fence us
2,33777,!qscreen fence without holes,us,B001OJXVKW,Windscreen4less Heavy Duty Privacy Screen Fence in Color Solid Black 6' x 50' Brass Grommets 150 GSM - Customized,,"Product measures exactly 50' long x 5'8"" tall, perfect fit for a fence that’s 6 ft tall.\nConstructed of new high density material- polyethylene, allows maximum air to go through.\nPrevents harmful UV sun lights from damaging for up to 90%, and fabric has a 2.5” enforced binding, so it’s able to hold up even in extreme weather condition.\n2 grommets on each side of fabric to ensure strength, and it’s easy to install.\nHas a 3 Year Limited Warranty under normal usage and weather.",Windscreen4less,Black,us,!qscreen fence without holes us,Windscreen4less Heavy Duty Privacy Screen Fence in Color Solid Black 6' x 50' Brass Grommets 150 GSM - Customized us
3,33777,!qscreen fence without holes,us,B0030E1WLE,"Richell Freestanding Pet Gate, Large, Origami White",,"Freestanding gate with side panels for safely confining pets\nNo installation necessary; adjustable width from 39.8 to 71.3 inches\nWide base prevents tipping; rubber feet protect floors; folds for storing\nLow enough for people to stride over; beautiful hardwood finish. Made from Rubberwood\nMeasures 39.8 to 71.3 by 17.7 by 20.1 inches. 1.25 inch space between wire slats, Wire is 7 gage or 0.144 inch",Richell,Origami White,us,!qscreen fence without holes us,"Richell Freestanding Pet Gate, Large, Origami White us"
4,33777,!qscreen fence without holes,us,B003VNKLIY,"Regalo 192-Inch Super Wide Adjustable Baby Gate and Play Yard, 4-In-1, Bonus Kit, Includes 4 Pack of Wall Mounts",,"SUPERWIDE: Fits opens up to 192 inches wide and stands 28 inches tall\n2-IN-1: Can convert from a super-wide gate to an 8-panel play yard. No tools require, quick and easy. Includes 4 pack of wall mounts\nCONFIGURABLE: Each panel can be adjusted to fit almost any opening. Great for wide spaces, angled openings, hallway, doorway, bottom of stairs or barrier. Folds flat for storage.Wipes clean\nDURABLE AND CONVENIENT: The all steel design is durable and convenient with a walk-through door\nSAFETY: Designed with a walk-through door with safety locking feature. Wall mounted for added security and protection. Meets all current safety standards. For children up to 24 months and great for pets too",Regalo,White,us,!qscreen fence without holes us,"Regalo 192-Inch Super Wide Adjustable Baby Gate and Play Yard, 4-In-1, Bonus Kit, Includes 4 Pack of Wall Mounts us"


In [13]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 KB 290.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 KB 1.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.1-py3-none-any.whl size=125774 sha256=7cf3aed7eef75283e341120833a9e8b44be458e7da849cc2bfbd7ebe9e4d65d3
  Stored in directory: /root/.cache/pip/wheels/58/27/2f/708b4f002c226e57b6243769da345c650633175c7634f93365
Successfully built sentence-transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.5.1
    Uninstalling huggingface-hub-0.5.1:
      Successfully uninstalled huggingface-hub-0.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cached-path 1.1.2 requires huggingface-hub<0.6.0,>=0.0.12, but you have huggingface-hub 0.8.1 which i

In [14]:
from sentence_transformers import SentenceTransformer, util

In [15]:
model = SentenceTransformer('../input/sentence-transformer-classification-model')

In [16]:
query_input_list = query_prod_test['query_input'].to_list()
product_input_list = query_prod_test['product_input'].to_list()

In [17]:
embeddings_query = model.encode(query_input_list, convert_to_tensor=True)
embeddings_prod = model.encode(product_input_list, convert_to_tensor=True)

Batches:   0%|          | 0/5244 [00:00<?, ?it/s]

Batches:   0%|          | 0/5244 [00:00<?, ?it/s]

In [18]:
cosine_scores = util.pairwise_cos_sim(embeddings_query, embeddings_prod)

In [19]:
cosine_scores.shape

torch.Size([167799])

In [20]:
query_prod_test['score'] = cosine_scores.tolist()

In [21]:
query_prod_test.head()

,query_id,query,query_locale,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale,query_input,product_input,score
0,33777,!qscreen fence without holes,us,B000HMCA7W,"Tenax 60041989 Multi-Purpose Net, 3' x 50', Black",,"Mesh .70""X.98""\nVirtually invisible\nWill not rust or corrode\nAvailable in black",Tenax,Black,us,!qscreen fence without holes us,"Tenax 60041989 Multi-Purpose Net, 3' x 50', Black us",0.308208
1,33777,!qscreen fence without holes,us,B0019Y6TTC,Garden Creations JB4710 Extendable Instant Fence,,"EXPANDABLE FLEXI-SCREEN stretch your fence to the width compatable with your home\nESTHETIC DESIGN easily place your fence inside or outdoors, wood finish is a perfect match for interior home designs as well as porch finishes\nPLANT DIVIDER perfect for flowerbeds, easily separate your plants while maintaining visual garden appeal\nPORTABLE light weight design, easy to move from room to room\nDURABLE fence stands on two wooden platforms, no digging or support needed","Creation's Garden Natural Products, Inc",Brown,us,!qscreen fence without holes us,Garden Creations JB4710 Extendable Instant Fence us,0.436759
2,33777,!qscreen fence without holes,us,B001OJXVKW,Windscreen4less Heavy Duty Privacy Screen Fence in Color Solid Black 6' x 50' Brass Grommets 150 GSM - Customized,,"Product measures exactly 50' long x 5'8"" tall, perfect fit for a fence that’s 6 ft tall.\nConstructed of new high density material- polyethylene, allows maximum air to go through.\nPrevents harmful UV sun lights from damaging for up to 90%, and fabric has a 2.5” enforced binding, so it’s able to hold up even in extreme weather condition.\n2 grommets on each side of fabric to ensure strength, and it’s easy to install.\nHas a 3 Year Limited Warranty under normal usage and weather.",Windscreen4less,Black,us,!qscreen fence without holes us,Windscreen4less Heavy Duty Privacy Screen Fence in Color Solid Black 6' x 50' Brass Grommets 150 GSM - Customized us,0.397781
3,33777,!qscreen fence without holes,us,B0030E1WLE,"Richell Freestanding Pet Gate, Large, Origami White",,"Freestanding gate with side panels for safely confining pets\nNo installation necessary; adjustable width from 39.8 to 71.3 inches\nWide base prevents tipping; rubber feet protect floors; folds for storing\nLow enough for people to stride over; beautiful hardwood finish. Made from Rubberwood\nMeasures 39.8 to 71.3 by 17.7 by 20.1 inches. 1.25 inch space between wire slats, Wire is 7 gage or 0.144 inch",Richell,Origami White,us,!qscreen fence without holes us,"Richell Freestanding Pet Gate, Large, Origami White us",0.440367
4,33777,!qscreen fence without holes,us,B003VNKLIY,"Regalo 192-Inch Super Wide Adjustable Baby Gate and Play Yard, 4-In-1, Bonus Kit, Includes 4 Pack of Wall Mounts",,"SUPERWIDE: Fits opens up to 192 inches wide and stands 28 inches tall\n2-IN-1: Can convert from a super-wide gate to an 8-panel play yard. No tools require, quick and easy. Includes 4 pack of wall mounts\nCONFIGURABLE: Each panel can be adjusted to fit almost any opening. Great for wide spaces, angled openings, hallway, doorway, bottom of stairs or barrier. Folds flat for storage.Wipes clean\nDURABLE AND CONVENIENT: The all steel design is durable and convenient with a walk-through door\nSAFETY: Designed with a walk-through door with safety locking feature. Wall mounted for added security and protection. Meets all current safety standards. For children up to 24 months and great for pets too",Regalo,White,us,!qscreen fence without holes us,"Regalo 192-Inch Super Wide Adjustable Baby Gate and Play Yard, 4-In-1, Bonus Kit, Includes 4 Pack of Wall Mounts us",0.360977


In [22]:
output = []
for query_id, _df in query_prod_test.groupby(['query_id']):
    output.append(_df.sort_values(by='score', ascending=False)[['query_id', 'query_locale', 'product_id', 'product_locale']])

In [23]:
output_df = pd.concat(output, ignore_index=True)

In [24]:
output_df.shape

(167799, 4)

In [25]:
output_df.to_csv('./output_sentence_transformers_baseline.csv', index=False)